In [ ]:
import pandas as pd
import glob

# Dimensão acesso

In [ ]:
files = glob.glob('../Dados/Raw/Dimensão Uso/*.csv')
bacen = pd.DataFrame(columns=['Date'])

for file in files:
    df = pd.read_csv(file, sep=";", encoding='latin-1', decimal = '.', thousands = ',')
    bacen = bacen.merge(df, how='outer', left_on='Date', right_on='Date')

In [ ]:
bacen = bacen.drop([16])
regioes = ['Central-West','Northeast','North ','Southeast','South ']
troca_nomes = pd.read_excel('../Dados/Raw/Dimensão Uso/Depara.xlsx')

In [ ]:
dataframe = pd.DataFrame()

for regiao in regioes:
    q = bacen[bacen.columns[bacen.columns.str.contains(regiao)] | bacen.columns[bacen.columns.str.contains('Date')]]
    columns = q.columns

    for c in columns:
        for i,x in troca_nomes.iterrows():
            if (x['parte importante'] in c):
                q = q.rename(columns={c:x['novo nome']})
                break
                
    q['Região'] = regiao
    dataframe = dataframe.append(q)

In [ ]:
dataframe = dataframe.reset_index(drop = True)
dataframe = dataframe.apply(lambda x: x.str.replace(',',''))

for c in dataframe.columns:
    dataframe.loc[dataframe[c] == '-', c] = None
    
dataframe['Adults with a banking relationship (million)'] = dataframe['Adults with a banking relationship (million)'].astype(float)

## Fintechs

In [ ]:
fint = pd.read_csv('../Dados/Raw/Fintechs/fintechs.csv', sep=",", encoding='latin-1', decimal = ',', thousands = '.')

dataframe.Date = dataframe.Date.astype(int) # modificar tipo de dado da coluna Date

fint = fint.rename(columns = {'Ano' : 'Date'}) # renomear coluna

dataframe = dataframe.merge(fint, left_on = 'Date', right_on = 'Date', how = 'outer')

## Area Regiao

In [ ]:
area = pd.read_excel('../Dados/Raw/Area Regiao/area_regiao.xlsx')

dataframe.loc[dataframe['Região'] == 'South ', 'Região'] = 'South'
dataframe.loc[dataframe['Região'] == 'North ', 'Região'] = 'North'

dataframe = dataframe.merge(area, how = 'outer')

## Populacao Regiao

In [ ]:
pop = pd.read_excel('../Dados/Raw/Populacao/populacao_ibge_regiao (1).xlsx')

pop = pd.melt(pop, id_vars = ['Região']).rename(columns = {'Região' : 'Date', 'variable' : 'Região', 'value' : 'População'})

dataframe = dataframe.merge(pop, how = 'outer')

## Medidas Acesso

In [ ]:
columns = ['Number of headquarters of authorized financial institutions',
       'Number of branches of financial institutions',
       'Number of banking service outposts of financial institutions',
       'Number of electronic service outposts of financial institutions',
       'Number of correspondents of financial institutions ',
       'Number of ATMs of financial institutions',
       'Number of POSs of financial institutions',
       'Number of authorized credit unions ',
       'Number of credit unions banking service outposts',
       'Number of credit unions electronic service outposts',
       'Number of credit unions correspondents',
       'Number of branches of finance companies',
       'Number of banking services outpost of finance companies',
       'Number of electronic service outposts of finance companies',
       'Number of microentrepreneur, and small business credit companies correspondents',
       'Total invesment activity', 'Number Fintechs Brazil']

dataframe[columns] = dataframe[columns].astype(float)

## Geografico

In [ ]:
dfGeografico = dataframe.copy()
dfGeografico[columns] = dfGeografico[columns] * 1000
for c in columns:
    dfGeografico[c] = dfGeografico[c] / dfGeografico['Área']

## Demográfico

In [ ]:
dfDemografico = dataframe.copy()
dfDemografico[columns] = dfDemografico[columns] * 10000
for c in columns:
    dfDemografico[c] = dfDemografico[c] / dfDemografico['População']

# Dimensão Uso

In [ ]:
dfUso = dataframe[['Região', 'Date', 'População', 'Adults with a banking relationship (million)']]

dfUso['Adults with a banking relationship (million)'] = dfUso['Adults with a banking relationship (million)'] * 1000000

dfUso['Bancarização'] = dfUso.apply(lambda row : row['Adults with a banking relationship (million)'] / row['População'], axis = 1)

# PIB

In [ ]:
names = ['Date', 'Central-West', 'Northeast', 'North', 'Southeast', 'South']

path = '../Dados/Raw/PIB/PIB 2010 A 2017.xlsx'

pib = pd.read_excel(path, parse_dates = ['Date'], names = names, header = 0)

In [ ]:
pib.Date = pib.Date.dt.year
pib = pd.melt(pib, id_vars = ['Date']).rename(columns = {'value' : 'PIB', 'variable' : 'Região'})

In [ ]:
dfUso = dfUso.merge(pib)

## Depósito - Crédito

In [ ]:
depCred = pd.read_csv('../Dados/Raw/Depositos/Consolidado.csv', sep=",", encoding='latin-1', header = 0) 

columnsDep = ['VERBETE_420_DEPOSITOS_DE_POUPANCA',
           'VERBETE_430_DEPOSITOS_INTERIFNANCEIROS',
           'VERBETE_431_DEPOSITOS_INTERFINANCEIROS',
           'VERBETE_432_DEPOSITOS_A_PRAZO',
           'VERBETE_112_DEPOSITOS_BANCARIOS']

columnsCred = ['VERBETE_160_OPERACOES_DE_CREDITO', 'VERBETE_172_OUTROS_CREDITOS']

depCred['Depósito'] = 0
depCred['Crédito' ] = 0

for c in columnsDep:
    depCred['Depósito'] = depCred['Depósito'] + depCred[c]

for c in columnsCred:
    depCred['Crédito'] = depCred['Crédito'] + depCred[c]

In [ ]:
depCred = depCred.rename(columns = {'#DATA_BASE' : 'Date'})
depCred = depCred[['Date', 'UF', 'Crédito', 'Depósito']]

depCred.Date = depCred.Date.astype(int).astype(str)

depCred['Year']  = depCred.Date.apply(lambda date : date[0:4])
depCred['Month'] = depCred.Date.apply(lambda date : date[4:6])

depCred = depCred.drop(columns = ['Date'])
depCred = depCred.groupby(['UF', 'Year']).sum().reset_index()

In [ ]:
estadoRegiao = pd.read_csv('../Dados/Raw/EstadoRegiao/Sigla_estado.csv', encoding = 'latin_1')
estadoRegiao = estadoRegiao.rename(columns = {'Sigla' : 'UF'})

depCred = depCred.merge(estadoRegiao)
depCred = depCred.drop(columns = {'UF'})
depCred = depCred.rename(columns = {'Year' : 'Date'})
depCred = depCred.groupby(['Região', 'Date']).sum().reset_index()
depCred.Date = depCred.Date.astype(int)

In [ ]:
dfUso = dfUso.merge(depCred)

## Desinflacionar (Irrelevante)

In [ ]:
ipca = pd.read_excel('../Dados/Raw/IPCA/ipca_2000_2019.xlsx')
ipca = ipca.rename(columns={'Ano':'Date',100:'Taxa IPCA'})
ipca = ipca[['Date','Taxa IPCA']]

taxa2019 = ipca.loc[ipca.Date == 2019, 'Taxa IPCA'].values[0]

In [ ]:
dfUso = dfUso.merge(ipca)

dfUso['PIB'     ] = dfUso['PIB'     ] * taxa2019 / dfUso['Taxa IPCA']
dfUso['Crédito' ] = dfUso['Crédito' ] * taxa2019 / dfUso['Taxa IPCA']
dfUso['Depósito'] = dfUso['Depósito'] * taxa2019 / dfUso['Taxa IPCA']

## Nível de Crédito e Depósito

In [ ]:
dfUso['Nível Crédito'] = dfUso['Crédito'] / dfUso['PIB']

In [ ]:
dfUso['Nível Depósito'] = dfUso['Depósito'] / dfUso['PIB']

# Exportar

In [ ]:
dfDemografico.to_csv('../Dados/Processed/Acesso/Demografico.csv', index = None)
dfGeografico .to_csv( '../Dados/Processed/Acesso/Geofrafico.csv', index = None)

dfUso.to_csv('../Dados/Processed/Uso/Consolidado.csv', index = None)